ライブラリのインポート

In [20]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import datetime as dt
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_log_error
import lightgbm as lgb
import numpy as np
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import KFold
import copy
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor

パスの設定

In [21]:
# Train Data
train_data_pass = '../data/raw_data/train.csv'

# Test Data
test_data_pass = '../data/raw_data/test.csv'

データの確認

In [22]:
# Train dataの読み込み
df_train = pd.read_csv(train_data_pass, parse_dates=['datetime'])

# 'datetime'列をdatetime型に変換
df_train['datetime'] = pd.to_datetime(df_train['datetime'])

# 指定位置に年，月，日，時間の列を挿入
df_train.insert(1, 'year', df_train['datetime'].dt.year)
df_train.insert(2, 'month', df_train['datetime'].dt.month)
df_train.insert(3, 'day', df_train['datetime'].dt.day)
df_train.insert(4, 'hour', df_train['datetime'].dt.hour)

# 結果を表示
df_train

,datetime,year,month,day,hour,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,2011,1,1,0,1,0,0,1,9.84,14.395,81,0.0000,3,13,16
1,2011-01-01 01:00:00,2011,1,1,1,1,0,0,1,9.02,13.635,80,0.0000,8,32,40
2,2011-01-01 02:00:00,2011,1,1,2,1,0,0,1,9.02,13.635,80,0.0000,5,27,32
3,2011-01-01 03:00:00,2011,1,1,3,1,0,0,1,9.84,14.395,75,0.0000,3,10,13
4,2011-01-01 04:00:00,2011,1,1,4,1,0,0,1,9.84,14.395,75,0.0000,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10881,2012-12-19 19:00:00,2012,12,19,19,4,0,1,1,15.58,19.695,50,26.0027,7,329,336
10882,2012-12-19 20:00:00,2012,12,19,20,4,0,1,1,14.76,17.425,57,15.0013,10,231,241
10883,2012-12-19 21:00:00,2012,12,19,21,4,0,1,1,13.94,15.910,61,15.0013,4,164,168
10884,2012-12-19 22:00:00,2012,12,19,22,4,0,1,1,13.94,17.425,61,6.0032,12,117,129


In [23]:
# Test dataの読み込み
df_test = pd.read_csv(test_data_pass, parse_dates=['datetime'])

# 'datetime'列をdatetime型に変換
df_test['datetime'] = pd.to_datetime(df_test['datetime'])

# 指定位置に年，月，日，時間の列を挿入
df_test.insert(1, 'year', df_test['datetime'].dt.year)
df_test.insert(2, 'month', df_test['datetime'].dt.month)
df_test.insert(3, 'day', df_test['datetime'].dt.day)
df_test.insert(4, 'hour', df_test['datetime'].dt.hour)

# 結果を表示
df_test

,datetime,year,month,day,hour,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
0,2011-01-20 00:00:00,2011,1,20,0,1,0,1,1,10.66,11.365,56,26.0027
1,2011-01-20 01:00:00,2011,1,20,1,1,0,1,1,10.66,13.635,56,0.0000
2,2011-01-20 02:00:00,2011,1,20,2,1,0,1,1,10.66,13.635,56,0.0000
3,2011-01-20 03:00:00,2011,1,20,3,1,0,1,1,10.66,12.880,56,11.0014
4,2011-01-20 04:00:00,2011,1,20,4,1,0,1,1,10.66,12.880,56,11.0014
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6488,2012-12-31 19:00:00,2012,12,31,19,1,0,1,2,10.66,12.880,60,11.0014
6489,2012-12-31 20:00:00,2012,12,31,20,1,0,1,2,10.66,12.880,60,11.0014
6490,2012-12-31 21:00:00,2012,12,31,21,1,0,1,1,10.66,12.880,60,11.0014
6491,2012-12-31 22:00:00,2012,12,31,22,1,0,1,1,10.66,13.635,56,8.9981


Train Dataをcasualとregisteredにデータを分割

In [24]:
# 指定されたカラムのみを抽出
df_train_casual = df_train[['datetime', 'year', 'month', 'day', 'hour', 'season', 'holiday', 'workingday', 'weather', 'temp', 'atemp', 'humidity', 'windspeed', 'casual']]
df_train_registered = df_train[['datetime', 'year', 'month', 'day', 'hour', 'season', 'holiday', 'workingday', 'weather', 'temp', 'atemp', 'humidity', 'windspeed', 'registered']]
df_train_count = df_train[['datetime', 'year', 'month', 'day', 'hour', 'season', 'holiday', 'workingday', 'weather', 'temp', 'atemp', 'humidity', 'windspeed', 'count']]

print("df_train_casual")
display(df_train_casual.head(2))

print("df_train_registered")
display(df_train_registered.head(2))

print("df_train_count")
display(df_train_count.head(2))

df_train_casual


,datetime,year,month,day,hour,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual
0,2011-01-01 00:00:00,2011,1,1,0,1,0,0,1,9.84,14.395,81,0.0,3
1,2011-01-01 01:00:00,2011,1,1,1,1,0,0,1,9.02,13.635,80,0.0,8


df_train_registered


,datetime,year,month,day,hour,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,registered
0,2011-01-01 00:00:00,2011,1,1,0,1,0,0,1,9.84,14.395,81,0.0,13
1,2011-01-01 01:00:00,2011,1,1,1,1,0,0,1,9.02,13.635,80,0.0,32


df_train_count


,datetime,year,month,day,hour,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count
0,2011-01-01 00:00:00,2011,1,1,0,1,0,0,1,9.84,14.395,81,0.0,16
1,2011-01-01 01:00:00,2011,1,1,1,1,0,0,1,9.02,13.635,80,0.0,40


weather == 4 のデータを3に変換

In [25]:
def change_weather(df):
    modified_rows = df[df["weather"] == 4]  # weather が 4 の行を抽出
    modified_datetimes = modified_rows["datetime"]  # 該当する datetime 列を取得

    # weather を 4 から 3 に置換
    df.loc[:, "weather"] = df["weather"].replace(4, 3)
    
    # 変更された datetime を表示
    if not modified_datetimes.empty:
        print("Changed datetime rows:")
        print(modified_datetimes)
    else:
        print("No rows were changed.")
    
    return df

df_train_casual = change_weather(df_train_casual)
df_train_registered = change_weather(df_train_registered)
df_train_count = change_weather(df_train_count)
df_test = change_weather(df_test)

display(df_train_casual.head(2))

Changed datetime rows:
5631   2012-01-09 18:00:00
Name: datetime, dtype: datetime64[ns]
Changed datetime rows:
5631   2012-01-09 18:00:00
Name: datetime, dtype: datetime64[ns]
Changed datetime rows:
5631   2012-01-09 18:00:00
Name: datetime, dtype: datetime64[ns]
Changed datetime rows:
154    2011-01-26 16:00:00
3248   2012-01-21 01:00:00
Name: datetime, dtype: datetime64[ns]


,datetime,year,month,day,hour,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual
0,2011-01-01 00:00:00,2011,1,1,0,1,0,0,1,9.84,14.395,81,0.0,3
1,2011-01-01 01:00:00,2011,1,1,1,1,0,0,1,9.02,13.635,80,0.0,8


weatherの移動平均を新たな特徴として追加

In [26]:
def add_weather_moving_average(df, window_size):
    # 既に "weather_ma" が存在する場合，何もしない
    if "weather_ma" in df.columns:
        return df
    
    # 移動平均を計算
    moving_avg = df["weather"].rolling(window=window_size, min_periods=1).mean()
    # 元の weather カラムの次の位置を取得
    weather_col_index = df.columns.get_loc("weather")
    # 新しいカラムを挿入
    df.insert(weather_col_index + 1, "weather_ma", moving_avg)
    return df

# ウィンドウサイズを指定して各データフレームに適用
window_size = 4  # 移動平均のウィンドウサイズ
df_train_casual = add_weather_moving_average(df_train_casual, window_size)
df_train_registered = add_weather_moving_average(df_train_registered, window_size)
df_train_count = add_weather_moving_average(df_train_count, window_size)
df_test = add_weather_moving_average(df_test, window_size)

# 確認
display(df_train_casual.head(2))

,datetime,year,month,day,hour,season,holiday,workingday,weather,weather_ma,temp,atemp,humidity,windspeed,casual
0,2011-01-01 00:00:00,2011,1,1,0,1,0,0,1,1.0,9.84,14.395,81,0.0,3
1,2011-01-01 01:00:00,2011,1,1,1,1,0,0,1,1.0,9.02,13.635,80,0.0,8


In [27]:
def add_weather_daily_moving_average(df, window_size):
    # 既に "weather_ma_day" が存在する場合，何もしない
    if "weather_ma_day" in df.columns:
        return df
    
    # datetimeを日付形式に変換（時間を無視して日単位にまとめる）
    df["date"] = pd.to_datetime(df["datetime"]).dt.date
    
    # 日単位で天気の平均値を計算
    daily_weather = df.groupby("date")["weather"].mean()
    
    # 日単位の移動平均を計算
    daily_moving_avg = daily_weather.rolling(window=window_size, min_periods=1).mean()
    
    # 日単位移動平均を元のデータフレームにマージ
    df = df.merge(daily_moving_avg.rename("weather_ma_day"), how="left", left_on="date", right_index=True)
    
    # "weather_ma_day" を "weather_ma" の次の列に挿入
    if "weather_ma" in df.columns:
        weather_ma_index = df.columns.get_loc("weather_ma")
        weather_ma_day_col = df.pop("weather_ma_day")  # 一旦列を削除
        df.insert(weather_ma_index + 1, "weather_ma_day", weather_ma_day_col)  # "weather_ma" の次に挿入
    else:
        # "weather_ma" が存在しない場合、最後の列に "weather_ma_day" を追加
        weather_ma_day_col = df.pop("weather_ma_day")
        df.insert(len(df.columns), "weather_ma_day", weather_ma_day_col)
    
    # 不要な "date" 列を削除
    df.drop(columns=["date"], inplace=True)
    
    return df

# ウィンドウサイズを指定して各データフレームに適用
window_size = 3  # 移動平均のウィンドウサイズ（日単位）
# df_train_casual = add_weather_daily_moving_average(df_train_casual, window_size)
# df_train_registered = add_weather_daily_moving_average(df_train_registered, window_size)
# df_train_count = add_weather_daily_moving_average(df_train_count, window_size)
# df_test = add_weather_daily_moving_average(df_test, window_size)

# 確認
display(df_train_casual.head(2))

,datetime,year,month,day,hour,season,holiday,workingday,weather,weather_ma,temp,atemp,humidity,windspeed,casual
0,2011-01-01 00:00:00,2011,1,1,0,1,0,0,1,1.0,9.84,14.395,81,0.0,3
1,2011-01-01 01:00:00,2011,1,1,1,1,0,0,1,1.0,9.02,13.635,80,0.0,8


In [28]:
def add_temp_moving_average(df, window_size):
    # 既に "temp_ma" が存在する場合，何もしない
    if "temp_ma" in df.columns:
        return df
    
    # 移動平均を計算
    moving_avg = df["temp"].rolling(window=window_size, min_periods=1).mean()
    # 元の temp カラムの次の位置を取得
    temp_col_index = df.columns.get_loc("temp")
    # 新しいカラムを挿入
    df.insert(temp_col_index + 1, "temp_ma", moving_avg)
    return df

# ウィンドウサイズを指定して各データフレームに適用
window_size = 4  # 移動平均のウィンドウサイズ
df_train_casual = add_temp_moving_average(df_train_casual, window_size)
df_train_registered = add_temp_moving_average(df_train_registered, window_size)
df_train_count = add_temp_moving_average(df_train_count, window_size)
df_test = add_temp_moving_average(df_test, window_size)

# 確認
display(df_train_casual.head(2))

,datetime,year,month,day,hour,season,holiday,workingday,weather,weather_ma,temp,temp_ma,atemp,humidity,windspeed,casual
0,2011-01-01 00:00:00,2011,1,1,0,1,0,0,1,1.0,9.84,9.84,14.395,81,0.0,3
1,2011-01-01 01:00:00,2011,1,1,1,1,0,0,1,1.0,9.02,9.43,13.635,80,0.0,8


曜日の追加　0（月曜日）～ 6（日曜日）

In [29]:
def add_weekday_column(df):
    # 既に "weekday" が存在する場合，何もしない
    if "weekday" in df.columns:
        return df
    
    # 曜日を計算
    df = df.copy()  # 必要に応じて元のDataFrameを保護
    df.loc[:, 'weekday'] = pd.to_datetime(df[['year', 'month', 'day']]).dt.dayofweek  # 0: 月曜日, 6: 日曜日

    # 'hour' カラムの次に 'weekday' を挿入
    hour_col_index = df.columns.get_loc('hour')
    weekday_values = df.pop('weekday')
    df.insert(hour_col_index + 1, 'weekday', weekday_values)

    return df


df_train_casual = add_weekday_column(df_train_casual)
df_train_registered = add_weekday_column(df_train_registered)
df_train_count = add_weekday_column(df_train_count)
df_test = add_weekday_column(df_test)

# 確認
display(df_train_casual.head(2))

,datetime,year,month,day,hour,weekday,season,holiday,workingday,weather,weather_ma,temp,temp_ma,atemp,humidity,windspeed,casual
0,2011-01-01 00:00:00,2011,1,1,0,5,1,0,0,1,1.0,9.84,9.84,14.395,81,0.0,3
1,2011-01-01 01:00:00,2011,1,1,1,5,1,0,0,1,1.0,9.02,9.43,13.635,80,0.0,8


モデルの定義　RandamForest & LightGBM

In [30]:
# RandomForestモデル（デフォルト設定）
RF_casual_model = RandomForestRegressor(random_state=42)
RF_registered_model = RandomForestRegressor(random_state=42)

# LightGBMモデル（デフォルト設定）
LGBM_casual_model = LGBMRegressor(random_state=42, verbosity=-1)
LGBM_registered_model = LGBMRegressor(random_state=42, verbosity=-1)

評価関数RMSLEと学習・評価関数の定義

In [31]:
# RMSLE評価関数
def evaluate_model_rmsle(y_true, y_pred):
    y_pred = np.where(y_pred < 0, 0, y_pred)
    return np.sqrt(mean_squared_log_error(y_true, y_pred))

# 学習と評価
def train_and_evaluate_model(data, model, drop_column, n_splits=5, tranlate_log=True):
    # 最終列をターゲット列として自動取得
    target_column = data.columns[-1]

    # ユニークな日を分割して各foldに分ける
    unique_days = data['day'].unique()
    day_groups = np.array_split(np.sort(unique_days), n_splits)
    # fold結果を出力
    print("Day Groups for each fold:")
    print(" | ".join([f"Fold-{i}: {', '.join(map(str, group))}" for i, group in enumerate(day_groups, start=1)]))

    scores = []
    saved_models = []
    pred_list = []

    for fold, days in enumerate(day_groups, start=1):
        train_data = data[~data['day'].isin(days)]
        valid_data = data[data['day'].isin(days)]

        # 訓練と検証のデータから指定された列を削除
        X_train, X_valid = train_data.drop(columns=["datetime", target_column] + drop_column), valid_data.drop(columns=["datetime", target_column] + drop_column)
        y_train, y_valid = train_data[target_column], valid_data[target_column]

        # モデルの学習
        model_fold = copy.deepcopy(model)
        if tranlate_log:
            model_fold.fit(X_train, np.log1p(y_train))
        else:
            model_fold.fit(X_train, y_train)

        # validデータに対する評価
        y_pred = model_fold.predict(X_valid)
        if tranlate_log:
            rmsle = evaluate_model_rmsle(y_valid, np.expm1(y_pred))
            pred_list.append(np.expm1(y_pred))
        else:
            rmsle = evaluate_model_rmsle(y_valid, y_pred)
            pred_list.append(y_pred)
        scores.append(rmsle)

        # 各モデルの保存
        saved_models.append({"model_name": f"{target_column}_fold_{fold}", "model": copy.deepcopy(model_fold)})

    print(f"{target_column.capitalize()} Prediction - Mean RMSLE: {np.mean(scores):.3f}")
    return saved_models, scores, pred_list


def count_combined_rmsle(casual_pred_list, registered_pred_list, df_train_count, n_splits=5):
    count_scores = []
    # ユニークな日を分割して各foldに分ける
    unique_days = df_train_count['day'].unique()
    day_groups = np.array_split(np.sort(unique_days), n_splits)

    for fold, days in enumerate(day_groups, start=1):
        # casualとregisteredの予測値を取得
        y_casual_pred = casual_pred_list[fold - 1]
        y_registered_pred = registered_pred_list[fold - 1]
        
        # 合算してcountを生成
        y_count_pred = y_casual_pred + y_registered_pred
        
        # テストデータのcount列
        y_count_true = df_train_count[df_train_count['day'].isin(days)]['count']
        
    # RMSLEの計算
    count_rmsle = evaluate_model_rmsle(y_count_true, y_count_pred)
    count_scores.append(count_rmsle)
    
    # 平均したRMSLEを出力
    print(f"\nCount Prediction - Mean RMSLE: {np.mean(count_scores):.3f}")
    return count_scores

学習

In [32]:
# fold数を指定
n_splits = 5

# 除外する説明変数を指定
drop_column = ['month']
#drop_column = []

# Random Forest
# print("-"*15, " RandomForest ", "-"*15)
# print("Training Casual Model")
# RF_casual_models, RF_casual_scores, RF_casual_pred_list = train_and_evaluate_model(df_train_casual, RF_casual_model, drop_column, n_splits)
# print("\nTraining Registered Model")
# RF_registered_models, RF_registered_scores, RF_registered_pred_list = train_and_evaluate_model(df_train_registered, RF_registered_model, drop_column, n_splits)
# RF_combined_scores = count_combined_rmsle(RF_casual_pred_list, RF_registered_pred_list, df_train_count, n_splits)# 合算したcountのRMSLEを計算

# LightGBM
print("\n","-"*15, " LightGBM ", "-"*15)
print("Training Casual Model")
LGBM_casual_models, LGBM_casual_scores, LGBM_casual_pred_list = train_and_evaluate_model(df_train_casual, LGBM_casual_model, drop_column, n_splits)
print("\nTraining Registered Model")
LGBM_registered_models, LGBM_registered_scores, LGBM_registered_pred_list = train_and_evaluate_model(df_train_registered, LGBM_registered_model, drop_column, n_splits)
LGBM_combined_scores = count_combined_rmsle(LGBM_casual_pred_list, LGBM_registered_pred_list, df_train_count, n_splits)# 合算したcountのRMSLEを計算


 ---------------  LightGBM  ---------------
Training Casual Model
Day Groups for each fold:
Fold-1: 1, 2, 3, 4 | Fold-2: 5, 6, 7, 8 | Fold-3: 9, 10, 11, 12 | Fold-4: 13, 14, 15, 16 | Fold-5: 17, 18, 19


Casual Prediction - Mean RMSLE: 0.509

Training Registered Model
Day Groups for each fold:
Fold-1: 1, 2, 3, 4 | Fold-2: 5, 6, 7, 8 | Fold-3: 9, 10, 11, 12 | Fold-4: 13, 14, 15, 16 | Fold-5: 17, 18, 19
Registered Prediction - Mean RMSLE: 0.304

Count Prediction - Mean RMSLE: 0.293


テストデータの予測

In [33]:
def predict_test_data(saved_casual_models, saved_registered_models, test_data, drop_column, output_file="submission.csv", translate_log=True):
    # `casual`と`registered`の予測値リスト
    casual_preds = []
    registered_preds = []

    # 各foldごとに予測を行い，平均を計算
    for fold in range(len(saved_casual_models)):
        # モデルの読み込み
        casual_model = saved_casual_models[fold]["model"]
        registered_model = saved_registered_models[fold]["model"]
        print(f"Using {saved_casual_models[fold]['model_name']} and {saved_registered_models[fold]['model_name']} for fold {fold + 1}")

        # テストデータの特徴量を取得
        X_test = test_data.drop(columns=["datetime"] + drop_column)

        # `casual`と`registered`の予測
        if translate_log:
            casual_preds.append(np.expm1(casual_model.predict(X_test)))
            registered_preds.append(np.expm1(registered_model.predict(X_test)))
        else:
            casual_preds.append(casual_model.predict(X_test))
            registered_preds.append(registered_model.predict(X_test))

    # 各foldの予測値を平均して最終予測値を計算
    casual_final_pred = np.mean(casual_preds, axis=0)
    registered_final_pred = np.mean(registered_preds, axis=0)
    count_final_pred = casual_final_pred + registered_final_pred

    # 結果をDataFrameにまとめる
    submission_df = pd.DataFrame({
        "datetime": test_data["datetime"],
        "count": count_final_pred
    })

    # CSVとして保存
    submission_df.to_csv(output_file, index=False)
    print(f"Submission file saved as {output_file}")

In [34]:
# テストデータの予測とCSV保存
#predict_test_data(RF_casual_models, RF_registered_models, df_test, drop_column, output_file="../out/submission-RF.csv")
predict_test_data(LGBM_casual_models, LGBM_registered_models, df_test, drop_column, output_file="../out/submission-LGBM.csv")

Using casual_fold_1 and registered_fold_1 for fold 1
Using casual_fold_2 and registered_fold_2 for fold 2
Using casual_fold_3 and registered_fold_3 for fold 3
Using casual_fold_4 and registered_fold_4 for fold 4
Using casual_fold_5 and registered_fold_5 for fold 5
Submission file saved as ../out/submission-LGBM.csv


### Optuna導入

In [35]:
import optuna
from tqdm import tqdm

# ログレベルをエラーのみに設定
optuna.logging.set_verbosity(optuna.logging.ERROR)

LightGBM

In [36]:
# 学習と評価
def train_and_evaluate_model_with_optuna(data, drop_column, n_splits=5, tranlate_log=True, params=None):
    target_column = data.columns[-1]
    unique_days = data['day'].unique()
    day_groups = np.array_split(np.sort(unique_days), n_splits)
    scores = []

    for fold, days in enumerate(day_groups, start=1):
        train_data = data[~data['day'].isin(days)]
        valid_data = data[data['day'].isin(days)]

        X_train, X_valid = train_data.drop(columns=["datetime", target_column] + drop_column), valid_data.drop(columns=["datetime", target_column] + drop_column)
        y_train, y_valid = train_data[target_column], valid_data[target_column]

        model = LGBMRegressor(random_state=42, verbosity=-1, **params)
        if tranlate_log:
            model.fit(X_train, np.log1p(y_train))
        else:
            model.fit(X_train, y_train)

        y_pred = model.predict(X_valid)
        if tranlate_log:
            rmsle = evaluate_model_rmsle(y_valid, np.expm1(y_pred))
        else:
            rmsle = evaluate_model_rmsle(y_valid, y_pred)
        scores.append(rmsle)

    return np.mean(scores)

In [37]:
# Optunaによるハイパーパラメータ探索
def objective(trial, data, drop_column, n_splits):
    params = {
        "num_leaves": trial.suggest_int("num_leaves", 20, 300),
        "max_depth": trial.suggest_int("max_depth", 3, 15),
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1, log=True),  
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0)
    }

    rmsle = train_and_evaluate_model_with_optuna(data, drop_column, n_splits, tranlate_log=True, params=params)
    return rmsle

# Optunaの実行
def optimize_model(data, drop_column, n_splits=5, n_trials=50):
    study = optuna.create_study(direction="minimize")
    pbar = tqdm(total=n_trials, desc="Optuna Optimization Progress")
    
    def objective_with_progress(trial):
        # プログレスバーを更新
        result = objective(trial, data, drop_column, n_splits)
        pbar.update(1)
        return result

    study.optimize(objective_with_progress, n_trials=n_trials)
    pbar.close()
    return study


In [38]:
print("-" * 15, " Optuna LightGBM ", "-" * 15)
print("Optimizing Casual Model")
casual_study = optimize_model(df_train_casual, drop_column, n_splits=n_splits, n_trials=50)
print("Best Parameters for Casual Model:", casual_study.best_params)

print("\nOptimizing Registered Model")
registered_study = optimize_model(df_train_registered, drop_column, n_splits=n_splits, n_trials=50)
print("Best Parameters for Registered Model:", registered_study.best_params)

# 最適化後のモデルで学習と評価
LGBM_casual_model = LGBMRegressor(random_state=42, verbosity=-1, **casual_study.best_params)
LGBM_registered_model = LGBMRegressor(random_state=42, verbosity=-1, **registered_study.best_params)

print("\nTraining Casual Model with Optimized Parameters")
LGBM_casual_models, LGBM_casual_scores, LGBM_casual_pred_list = train_and_evaluate_model(df_train_casual, LGBM_casual_model, drop_column, n_splits)
print("\nTraining Registered Model with Optimized Parameters")
LGBM_registered_models, LGBM_registered_scores, LGBM_registered_pred_list = train_and_evaluate_model(df_train_registered, LGBM_registered_model, drop_column, n_splits)
LGBM_combined_scores = count_combined_rmsle(LGBM_casual_pred_list, LGBM_registered_pred_list, df_train_count, n_splits)

---------------  Optuna LightGBM  ---------------
Optimizing Casual Model


Optuna Optimization Progress: 100%|██████████| 50/50 [03:17<00:00,  3.95s/it]


Best Parameters for Casual Model: {'num_leaves': 44, 'max_depth': 11, 'learning_rate': 0.009340038052550173, 'n_estimators': 793, 'min_child_samples': 45, 'subsample': 0.7684499269394517, 'colsample_bytree': 0.9607641409390035}

Optimizing Registered Model


Optuna Optimization Progress: 100%|██████████| 50/50 [02:39<00:00,  3.19s/it]


Best Parameters for Registered Model: {'num_leaves': 258, 'max_depth': 6, 'learning_rate': 0.01821679940509693, 'n_estimators': 782, 'min_child_samples': 48, 'subsample': 0.8976629892251838, 'colsample_bytree': 0.7359295987386981}

Training Casual Model with Optimized Parameters
Day Groups for each fold:
Fold-1: 1, 2, 3, 4 | Fold-2: 5, 6, 7, 8 | Fold-3: 9, 10, 11, 12 | Fold-4: 13, 14, 15, 16 | Fold-5: 17, 18, 19
Casual Prediction - Mean RMSLE: 0.505

Training Registered Model with Optimized Parameters
Day Groups for each fold:
Fold-1: 1, 2, 3, 4 | Fold-2: 5, 6, 7, 8 | Fold-3: 9, 10, 11, 12 | Fold-4: 13, 14, 15, 16 | Fold-5: 17, 18, 19
Registered Prediction - Mean RMSLE: 0.300

Count Prediction - Mean RMSLE: 0.292


In [39]:
predict_test_data(LGBM_casual_models, LGBM_registered_models, df_test, drop_column, output_file="../out/submission-LGBM-final.csv")

Using casual_fold_1 and registered_fold_1 for fold 1
Using casual_fold_2 and registered_fold_2 for fold 2
Using casual_fold_3 and registered_fold_3 for fold 3
Using casual_fold_4 and registered_fold_4 for fold 4
Using casual_fold_5 and registered_fold_5 for fold 5
Submission file saved as ../out/submission-LGBM-final.csv
